# Makemore Part 1: Bigram Character-Level Language Model

Building a character-level language model that generates name-like words. Two approaches:
1. **Counting-based**: Directly estimate bigram probabilities from data
2. **Neural network**: Learn the same probabilities via gradient descent

## Data Loading

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
words = open("names.txt", 'r').read().splitlines()
print(f'{len(words)} names, lengths {min(len(w) for w in words)}-{max(len(w) for w in words)}')
words[:10]

## Character Vocabulary

Map each character to an integer index. The special token `.` (index 0) represents both start and end of a name.

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}
itos

## Approach 1: Bigram Counting

Count how often each character follows another across all names, then normalize to get probabilities.

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)

for w in words:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        N[idx1, idx2] += 1

### Sampling from the Model

Add-one smoothing prevents zero probabilities for unseen bigrams. Then sample autoregressively until the end token `.` is produced.

In [ ]:
P = (N + 1).float()
P = P / P.sum(1, keepdim=True)

In [ ]:
g = torch.Generator().manual_seed(252525)

for i in range(15):
    out = []
    idx = 0
    while True:
        p = P[idx]
        idx = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[idx])
        if idx == 0:
            print(''.join(out))
            break

### Evaluating with Negative Log Likelihood

The quality metric: average negative log likelihood over the dataset. Lower is better. A perfect model that memorized the training set would have NLL = 0.

In [ ]:
log_like = 0.0
count = 0

for w in words:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        p = P[idx1, idx2]
        log_like += torch.log(p)
        count += 1

neg_ll = -log_like
norm_nll = neg_ll / count
print(f'Neg Logll = {neg_ll:.4f} | Norm Neg Logll = {norm_nll:.4f}')

## Approach 2: Neural Network (Gradient-Based)

Learn the same bigram probabilities using a single-layer neural network trained with gradient descent. The network takes a one-hot encoded character and outputs a probability distribution over the next character via softmax.

### Build the Dataset

In [ ]:
xs, ys = [], []

for w in words:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        xs.append(stoi[ch1])
        ys.append(stoi[ch2])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(f'{len(xs)} bigram examples')

### One-Hot Encoding

Each input character is represented as a 27-dimensional one-hot vector.

In [ ]:
xenc = F.one_hot(xs[:5], num_classes=27).float()
plt.imshow(xenc)
plt.title('One-hot encoding of first 5 bigram inputs')
plt.xlabel('Character index')
plt.ylabel('Example')
plt.show()

### Forward Pass

The forward pass: one-hot input x weight matrix -> logits -> exponentiate -> normalize (softmax). This is equivalent to learning a 27x27 probability table, the same structure as the counting approach.

In [ ]:
W = torch.randn((27, 27), requires_grad=True)

In [ ]:
xenc = F.one_hot(xs, num_classes=27).float()
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(len(ys)), ys].log().mean()
print(f'Initial loss: {loss.item():.4f}')

### Training Loop

In [ ]:
W = torch.randn((27, 27), requires_grad=True)

for i in range(100):
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss = -probs[torch.arange(len(ys)), ys].log().mean()

    W.grad = None
    loss.backward()
    W.data += -100 * W.grad

    if i % 20 == 0:
        print(f'Step {i:3d} | Loss = {loss.item():.4f}')

print(f'Final loss: {loss.item():.4f}')